In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from fredapi import Fred
import datetime
import joblib

# Initialize FRED API
fred = Fred(api_key='8445855393be6d75a6d33bcddffc7958')


In [2]:

# Define the series IDs for the required data 
series_ids = {
    '5-Year, 5-Year Forward Inflation Expectation Rate': 'T5YIFR',
    '1 Year Treasury Rate': 'DGS1',
    'fed_balance_sheet_assets': 'WALCL',
    'gdp': 'GDP',
    'Summary of Economic Projections: Longer-Term Federal Funds Rate': 'FEDTARMDLR',
    'US 10yr': 'DGS10',
    'RRP': "WLRRAL",
    'US Treasury General Account' : 'D2WLTGAL'
}

# Fetch the data starting from 11/8/2021
start_date = '2012-01-03'

# Download data and store in a dictionary
data = {name: fred.get_series(series_id, observation_start=start_date) for name, series_id in series_ids.items()}

# Convert the dictionary to a DataFrame
df = pd.DataFrame(data)

# Resample the monthly GDP to daily frequency using forward fill method
df = df.ffill()

# Calculate the Fed Balance Sheet Assets to GDP ratio
df['Fed BS Assets to GDP'] = df['fed_balance_sheet_assets'] / (df['gdp'] * 1000)

# Calculate Net Liquidity Indicator
df['Net Liquidity Indicator'] = df['fed_balance_sheet_assets'] - df['RRP'] - df['US Treasury General Account']

# Drop unnecessary columns
df = df.drop(['fed_balance_sheet_assets', 'gdp', 'RRP', 'US Treasury General Account'], axis=1)
df = df.dropna()


In [3]:
# Calculate percent changes over various time horizons\n",
time_horizons = [5, 20, 60]
for column in df.columns:
    if column not in ['Date', 'Target']:
        df[f'{column}_pct_change_5'] = df[column].pct_change(periods=5) * 100
        df[f'{column}_pct_change_20'] = df[column].pct_change(periods=20) * 100
        df[f'{column}_pct_change_60'] = df[column].pct_change(periods=60) * 100

In [26]:
# Define the target variable and the features
df['US 10yr lead'] = df['US 10yr'].shift(-20)
df = df.dropna()

X = df.drop(columns=['US 10yr lead'])
y = df['US 10yr lead']

# TimeSeriesSplit for cross-validation
initial_train_size = int(0.75 * y.shape[0])  # Set your desired initial train size here
tscv = TimeSeriesSplit(n_splits=df.shape[0] - initial_train_size, test_size=1)

# Define the XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [25, 50],
    'num_parallel_tree': [3,5],
    'max_depth': [5, 10],
    'learning_rate': [0.01,0.1],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8],
    'gamma': [0, 0.001]
}

In [27]:
# Initialize GridSearchCV with the XGBoost model and the parameter grid
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

# Store out-of-sample predictions
out_of_sample_preds = []

# TimeSeries cross-validation loop with GridSearchCV
for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # Perform GridSearchCV on the training data
    grid_search.fit(X_train, y_train)
    
    # Best parameters found by GridSearchCV
    print(f"Best Parameters: {grid_search.best_params_}")
    
    # Make predictions with the best model
    y_pred = grid_search.best_estimator_.predict(X_test)
    
    # Calculate performance metrics
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f'Mean Squared Error: {mse:.4f}')
    print(f'R2 Score: {r2:.4f}')
    
    # Store predictions
    out_of_sample_preds.append(y_pred)

# Convert the list of out-of-sample predictions to a DataFrame
out_of_sample_preds = np.concatenate(out_of_sample_preds)


KeyboardInterrupt: 

In [ ]:
# Plot predictions vs. actual values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, out_of_sample_preds, alpha=0.6)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Predictions vs Actual Values")
plt.show()

In [ ]:
import shap
# Initialize the SHAP explainer using TreeExplainer
explainer = shap.TreeExplainer(model)

# Compute SHAP values for the test set
shap_values = explainer.shap_values(X_test)

# Summary plot for overall feature importance
shap.summary_plot(shap_values, X_test)

In [ ]:
# Bar plot showing mean absolute SHAP values
shap.summary_plot(shap_values, X_test, plot_type="bar")

In [ ]:
# Select the last sample from the test set
last_sample = X_test[-1].reshape(1, -1)

# Compute SHAP values for this single prediction
shap_values_single = explainer.shap_values(last_sample)

# Force plot for a single prediction explanation
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values_single, last_sample)

In [ ]:

# Plot predictions vs. actual values (if desired)
plt.figure(figsize=(10, 6))
plt.plot(y.index, y.values, label='Actual Values')
plt.plot(y.index[-len(out_of_sample_preds):], out_of_sample_preds, label='Predicted Values', linestyle='--')
plt.xlabel('Date')
plt.ylabel('US 10yr')
plt.legend()
plt.show()

In [12]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.2, random_state=42)
# Convert data to DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train2, label=y_train2)
dtest = xgb.DMatrix(X_test2, label=y_test2)

In [23]:
# Define parameters for a Random Forest-like XGBoost model
params2 = {
    'booster': 'gbtree',  # Using gradient boosting trees (random forests are based on this)
    'objective': 'reg:squarederror',  # Regression task
    'num_parallel_tree': 1,  # Number of trees to train (this simulates the random forest size)
    'subsample': 0.8,  # Use 80% of the data for training each tree (random sampling)
    'colsample_bytree': 0.8,  # Randomly sample 80% of the features for each tree
    'seed': 42,  # Ensure reproducibility,
    'max_depth': 5  # Number of trees to train (this simulates the random forest size)

}

# Train the model
model2 = xgb.train(params2, dtrain)

# Make predictions
predictions2 = model2.predict(dtest)

# Evaluate performance (Mean Squared Error)
mse2 = mean_squared_error(y_test2, predictions2)
print(f'Mean Squared Error: {mse2}')

Mean Squared Error: 0.014791309888922384


In [24]:
params3 = {
    'booster': 'gbtree',  # Using gradient boosting trees (random forests are based on this)
    'objective': 'reg:squarederror',  # Regression task
    'num_parallel_tree': 50,  # Number of trees to train (this simulates the random forest size)
    'subsample': 0.8,  # Use 80% of the data for training each tree (random sampling)
    'colsample_bytree': 0.8,  # Randomly sample 80% of the features for each tree
    'seed': 42,  # Ensure reproducibility,
    'max_depth': 5  # Number of trees to train (this simulates the random forest size)

}

# Train the model
model3 = xgb.train(params3, dtrain)

# Make predictions
predictions3 = model3.predict(dtest)

# Evaluate performance (Mean Squared Error)
mse3 = mean_squared_error(y_test2, predictions3)
print(f'Mean Squared Error: {mse3}')

Mean Squared Error: 0.013303567826499128
